##### Configurações Iniciais

In [13]:
import os
import pandas as pd

from Modules.operation_analiser import OperationLoader, OperationAnalyzer
from Modules.forecast_loader import ForecastLoader
from Modules.variables_info import variables_info
from Modules.model_metrics import ModelPlotter
from Modules.scenario_classifier import RainScenarioClassifier

In [14]:
# Parâmetros gerais
lat_porto = -25.500511
lon_porto = -48.519331
forecast_path = '../Database/forecast_data'
paradas_path = '../Database/Parada_2024_chuva.xlsx'
timebox = ('202405', '202412')

variaveis = variables_info['variaveis']
legendas = variables_info['legendas']
legendas_dict = dict(zip(variaveis, legendas))

# Repositorios de Saída
base_output = '../Output/Analise_03'
os.makedirs('../Output/Analise_03', exist_ok=True)
for subfolder in ['Modelos', 'Tables', 'Imagens', 'Imagens/Modelos', 'Imagens/Modelos/Modelo_01', 'Imagens/Modelos/Modelo_02', 'Imagens/Metricas/Modelo_01', 'Imagens/Metricas/Modelo_02', 'Modelos/Modelo_01', 'Modelos/Modelo_02']:
    os.makedirs(os.path.join(base_output, subfolder), exist_ok=True)

##### Importação dos Dados para Analise

In [15]:
forecast = ForecastLoader(forecast_path, lat_target=lat_porto, lon_target=lon_porto)
df_forecast = forecast.load_forecasts(timebox, drop_duplicates=True)

operation_loader = OperationLoader(paradas_path)
df_paradas = operation_loader.load()

analyzer = OperationAnalyzer(df_paradas, df_forecast)
df_unificado = analyzer.marcar_paradas()

In [16]:
melhores = pd.read_csv(f'../Output/Analise_02/Tables/melhores_thresholds.csv')

#### Treinamento do Modelo Supervisionado para Classificação das Recomendações
##### Baseado nas Metricas

In [17]:
cenarios = {
    'conservador': melhores.sort_values('recall', ascending=False).head(3),
    'moderado': melhores.sort_values('f1_score', ascending=False).head(3),
    'arrojado': melhores.sort_values('precision', ascending=False).head(3),
}

modelo = RainScenarioClassifier(df_unificado, cenarios, base_output, legendas_dict)
modelo.aplicar_classificacao(modelo_nome="Modelo_01")


=== CENÁRIO: CONSERVADOR ===

RandomForest
              precision    recall  f1-score   support

           0       0.93      0.79      0.85      1116
           1       0.54      0.80      0.64       349

    accuracy                           0.79      1465
   macro avg       0.73      0.79      0.75      1465
weighted avg       0.83      0.79      0.80      1465

Matriz de Confusão: [[877 239]
 [ 70 279]]

LogisticRegression
              precision    recall  f1-score   support

           0       0.91      0.83      0.87      1116
           1       0.58      0.74      0.65       349

    accuracy                           0.81      1465
   macro avg       0.75      0.79      0.76      1465
weighted avg       0.83      0.81      0.82      1465

Matriz de Confusão: [[928 188]
 [ 89 260]]
Modelos salvos: ../Output/Analise_03/Modelos/Modelo_01/model_randomforest_conservador.pkl e ../Output/Analise_03/Modelos/Modelo_01/model_logisticregression_conservador.pkl

=== CENÁRIO: MODERADO =

##### Analise dos Modelos

In [18]:
# Carrega e prepara o DataFrame
df_metricas = pd.read_csv(f'{base_output}/Tables/metricas_modelo_01_por_cenario.csv')

plotter = ModelPlotter(df_metricas)

# 1. Comparação geral (sem filtro)
plotter.plotar_metricas(
    titulo='Desempenho Geral dos Modelos - Previsão de Paradas e Não-Paradas',
    output_path=f'{base_output}/Imagens/Modelos/Modelo_01/comparacao_geral_modelos.png'
)

# 2. Comparação por cenário - Classe: Parada
plotter.plotar_metricas(
    titulo='Comparação por Cenário - Classe: Parada',
    filtro_classe=1,
    col='cenario',
    output_path=f'{base_output}/Imagens/Modelos/Modelo_01/comparacao_por_cenario_parada.png'
)

# 3. Comparação por cenário - Classe: Não-Parada
plotter.plotar_metricas(
    titulo='Comparação por Cenário - Classe: Não-Parada',
    filtro_classe=0,
    col='cenario',
    output_path=f'{base_output}/Imagens/Modelos/Modelo_01/comparacao_por_cenario_nao_parada.png'
)

# 4. Comparação por cenário e por classe
plotter.plotar_metricas(
    titulo='Comparação por Cenário e Classe',
    col='cenario',
    row='classe',
    output_path=f'{base_output}/Imagens/Modelos/Modelo_01/comparacao_por_cenario_e_classe.png'
)

#### Treinamento do Modelo Supervisionado para Classificação das Recomendações
##### Combinação Variaveis (Estatistica + Metricas)

In [19]:
cenarios = {
    'conservador': [('precipitation_probability_55km', 15.0)],
    'moderado': [('precipitation_probability_35km', 25.0), ('lwe_precipitation_smooth_rate_maximum_35km', 0.5)],
    'arrojado': [('precipitation_probability_20km', 30.0), ('lwe_precipitation_smooth_rate_maximum_20km', 0.4)]
}

modelo = RainScenarioClassifier(df_unificado, cenarios, base_output, legendas_dict)
modelo.aplicar_classificacao(modelo_nome="Modelo_02")


=== CENÁRIO: CONSERVADOR ===

RandomForest
              precision    recall  f1-score   support

           0       0.92      0.82      0.87      1116
           1       0.58      0.78      0.67       349

    accuracy                           0.81      1465
   macro avg       0.75      0.80      0.77      1465
weighted avg       0.84      0.81      0.82      1465

Matriz de Confusão: [[918 198]
 [ 76 273]]

LogisticRegression
              precision    recall  f1-score   support

           0       0.92      0.82      0.87      1116
           1       0.58      0.78      0.67       349

    accuracy                           0.81      1465
   macro avg       0.75      0.80      0.77      1465
weighted avg       0.84      0.81      0.82      1465

Matriz de Confusão: [[918 198]
 [ 76 273]]
Modelos salvos: ../Output/Analise_03/Modelos/Modelo_02/model_randomforest_conservador.pkl e ../Output/Analise_03/Modelos/Modelo_02/model_logisticregression_conservador.pkl

=== CENÁRIO: MODERADO =

##### Analise dos Modelos

In [20]:
# Carrega e prepara o DataFrame
df_metricas = pd.read_csv(f'{base_output}/Tables/metricas_modelo_02_por_cenario.csv')

# Instancia o plotador com o DataFrame carregado
plotter = ModelPlotter(df_metricas)

# 1. Comparação geral (sem filtro)
plotter.plotar_metricas(
    titulo='Desempenho Geral dos Modelos - Previsão de Paradas e Não-Paradas',
    output_path=f'{base_output}/Imagens/Modelos/Modelo_02/comparacao_geral_modelos.png'
)

# 2. Comparação por cenário - Classe: Parada
plotter.plotar_metricas(
    titulo='Comparação por Cenário - Classe: Parada',
    filtro_classe=1,
    col='cenario',
    output_path=f'{base_output}/Imagens/Modelos/Modelo_02/comparacao_por_cenario_parada.png'
)

# 3. Comparação por cenário - Classe: Não-Parada
plotter.plotar_metricas(
    titulo='Comparação por Cenário - Classe: Não-Parada',
    filtro_classe=0,
    col='cenario',
    output_path=f'{base_output}/Imagens/Modelos/Modelo_02/comparacao_por_cenario_nao_parada.png'
)

# 4. Comparação por cenário e por classe
plotter.plotar_metricas(
    titulo='Comparação por Cenário e Classe',
    col='cenario',
    row='classe',
    output_path=f'{base_output}/Imagens/Modelos/Modelo_02/comparacao_por_cenario_e_classe.png'
)